# Chapter 25: Composing types

In [1]:
newtype Identity a = Identity { runIdentity :: a }

In [2]:
:t id
:k Identity

id :: forall a. a -> a

Identity :: * -> *

In [3]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

In [4]:
:t (.)
:k Compose

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

Compose :: (* -> *) -> (* -> *) -> * -> *

In [5]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

In [6]:
instance (Functor f, Functor g) => Functor (Compose f g) where
    fmap f (Compose fga) = Compose $ (fmap . fmap) f fga

When you compose two Functors, you get another Functor - it means that Functors are *closed under composition*.

---

In [10]:
import Control.Applicative (liftA2)

instance (Applicative f, Applicative g) => Applicative (Compose f g) where
    pure = Compose . pure . pure
    (Compose f) <*> (Compose a) = Compose $ liftA2 (<*>) f a

When you compose two Applicatives, you get another Applicative - it means that Applicatives are *closed under composition*.

---

Turns out, it is not true for Monads though (generally).

---

In [16]:
instance (Foldable f, Foldable g) => Foldable (Compose f g) where
    foldMap am (Compose fga) = foldMap (foldMap am) fga

In [22]:
instance (Traversable f, Traversable g) => Traversable (Compose f g) where
    traverse a2hb (Compose fga) = Compose <$> traverse (traverse a2hb) fga

---

In [33]:
class Bifunctor p where
    {-# MINIMAL bimap | first, second #-}
    
    bimap :: (a -> b) -> (c -> d) -> p a c -> p b d
    bimap f g = first f . second g
    
    first :: (a -> b) -> p a c -> p b c
    first f = bimap f id
    
    second :: (b -> c) -> p a b -> p a c
    second = bimap id

In [35]:
data Deux a b = Deux a b

instance Bifunctor Deux where
    bimap a2b c2d (Deux a c) = Deux (a2b a) (c2d c)

In [39]:
newtype Const a b = Const a

instance Bifunctor Const where
    bimap a2b _ (Const a) = Const (a2b a)

In [41]:
data Drei a b c = Drei a b c

instance Bifunctor (Drei x) where
    bimap a2b c2d (Drei x a c) = Drei x (a2b a) (c2d c)

In [44]:
data SuperDrei a b c = SuperDrei a b

instance Bifunctor (SuperDrei x) where
    bimap a2b _ (SuperDrei x a) = SuperDrei x (a2b a)

In [48]:
newtype SemiDrei a b c = SemiDrei a

instance Bifunctor (SemiDrei x) where
    bimap _ _ (SemiDrei x) = SemiDrei x

In [50]:
data Quadriceps a b c d = Quadzzz a b c d

instance Bifunctor (Quadriceps x y) where
    bimap a2b c2d (Quadzzz x y a c) = Quadzzz x y (a2b a) (c2d c)

In [51]:
data Either a b = Left a | Right b

instance Bifunctor Either where
    bimap a2b _ (Left a) = Left (a2b a)
    bimap _ c2d (Right c) = Right (c2d c)